In [1]:
reload_lamb()

# Variable free binding a la Jacobson

### Notebook author: Kyle Rawlins

This notebook implements (a bit of) the semantic side of Jacobson's account of binding and pronouns.  In so doing, we can get a better idea of how it works, as well as what work is carried by the pure type theory, and what work is carried by the syntactic side of things.  (I don't implement the syntax here.)

>Jacobson, Pauline. "[Towards a Variable-Free Semantics](http://link.springer.com/article/10.1023%2FA%3A1005464228727)", Linguistics and Philosophy 22, 1999. pp. 117-184.

The basic idea (in the end) is that rather than representing binding via indices, we can use type-shifts to hold out an argument slot where a pronoun appeared for later saturation.  I follow the development of the proposal in the article, by first using function composition.

(Warning: Because of the lack of syntax, this notebook is far from being a complete version of Jacobson's account!)

**Step 1**: define a function composition operation on meta-language functions (LFun objects)

* There are multiple ways of doing this.  I have chosen to construct a combinator of the correct type.  The function `geach_combinator` does this (see below for the output)
* This is effectively an implementation of the geach rule.

In [2]:
geach = %te L g_<Y,Z> : L f_<X,Y> : L x_X : g(f(x))

# use the Geach rule to implement function composition as a python function.
def fc_meta(g, f):
    return geach(g)(f).reduce_all()

Just as an example,here's what this combinator looks like for `X=e`, `Y=e`, `Z=t`, a case that will be common in the variable-free analysis of binding.

In [3]:
geach.try_adjust_type(tp("<<e,t>,<<e,e>,?>>"))

(λ g_<e,t>: (λ f_<e,e>: (λ x_e: g_<e,t>(f_<e,e>(x_e)))))

In [4]:
%%lamb 
f1 = L x_e : Cat(x)
f2 = L x_e : x_e

INFO (meta): Coerced guessed type t for 'Cat_t' into <e,t>, to match argument 'x_e'


${f1}_{\langle{}e,t\rangle{}}\:=\:\lambda{} x_{e} \: . \: {Cat}({x}_{e})$<br />
${f2}_{\langle{}e,e\rangle{}}\:=\:\lambda{} x_{e} \: . \: {x}_{e}$

In [5]:
%prun -s cumtime geach(f1)

In [6]:
t = tp("<<e,t>,X10>")
%prun -s cumtime r = geach.try_adjust_type(t)
r

(λ g_<e,t>: (λ f_<X,e>: (λ x_X: g_<e,t>(f_<X,e>(x_X)))))

In [7]:
fc_meta(f1,f2).derivation

In [8]:
# this trick is called "monkey patching".  It lets us use * for function composition in the metalanguage.
# note that it isn't really necessary here as function composition is now implemented as a part of the built in LFun class

# meta.LFun.__mul__ = fc_meta

In [9]:
f1 * f2

(λ x_e: Cat_<e,t>(x_e))

In [10]:
result = None
try:
    result = f2 * f1 # should generate a type mismatch
except types.TypeMismatch as e:
    result = e
result

lamb.types.TypeMismatch((λ x_e: x_e),
                        (λ x_e: Cat_<e,t>(x_e)),
                        'Function composition')

Next, we can use the Geach combinator to add a function composition operation to the composition system.  Internally, this does something like the above `fc_meta` function, but it is more straightforward to let the wrapper function `add_binary_rule` construct a composition rule from the combinator.

In [11]:
# add the FC rule to the composition system.  Note that we don't even need to use `fc_meta` here; can just pass the geach combinator in
system = lang.td_system.copy()
system.add_binary_rule(geach, "FC")
lang.set_system(system)
system

Composition system: H&K simple (copy)

**Step 2**: treat pronouns as identity functions, and allow type lifting for names.

(I have simply given the regular and type-lifted version of `Mary` under different names below.)

In [12]:
%%lamb
||him|| = L x_e  : x
||mary1|| = Mary_e
||mary2|| = L f_<e,t>: f(Mary_e)
||saw|| = L x_e : L y_e : Saw(y,x)

INFO (meta): Coerced guessed type t for 'Saw_t' into <(e,e),t>, to match argument '(y_e, x_e)'


$[\![\mathbf{\text{him}}]\!]^{}_{\langle{}e,e\rangle{}} \:=\: \lambda{} x_{e} \: . \: {x}_{e}$<br />
$[\![\mathbf{\text{mary1}}]\!]^{}_{e} \:=\: {Mary}_{e}$<br />
$[\![\mathbf{\text{mary2}}]\!]^{}_{\langle{}\langle{}e,t\rangle{},t\rangle{}} \:=\: \lambda{} f_{\langle{}e,t\rangle{}} \: . \: {f}_{\langle{}e,t\rangle{}}({Mary}_{e})$<br />
$[\![\mathbf{\text{saw}}]\!]^{}_{\langle{}e,\langle{}e,t\rangle{}\rangle{}} \:=\: \lambda{} x_{e} \: . \: \lambda{} y_{e} \: . \: {Saw}({y}_{e}, {x}_{e})$

In [13]:
mary2 * (saw * him)

CompositionResult(results=[⟦[mary2 [saw him]]⟧ = (λ x_e: Saw_<(e,e),t>(Mary_e, x_e))], failures=[⟦[mary2 [saw him]]⟧ = Type mismatch: '⟦mary2⟧ = (λ f_<e,t>: f_<e,t>(Mary_e))'/<<e,t>,t> and '⟦[saw him]⟧ = (λ x_e: (λ y_e: Saw_<(e,e),t>(y_e, x_e)))'/<e,<e,t>> conflict (mode: Function Application), ⟦[[saw him] mary2]⟧ = Type mismatch: '⟦[saw him]⟧ = (λ x_e: (λ y_e: Saw_<(e,e),t>(y_e, x_e)))'/<e,<e,t>> and '⟦mary2⟧ = (λ f_<e,t>: f_<e,t>(Mary_e))'/<<e,t>,t> conflict (mode: Function Application), ⟦[mary2 [saw him]]⟧ = Type mismatch: '⟦mary2⟧ = (λ f_<e,t>: f_<e,t>(Mary_e))'/<<e,t>,t> and '⟦[saw him]⟧ = (λ x_e: (λ y_e: Saw_<(e,e),t>(y_e, x_e)))'/<e,<e,t>> conflict (mode: Predicate Modification), ⟦[mary2 [saw him]]⟧ = Type mismatch: '⟦mary2⟧ = (λ f_<e,t>: f_<e,t>(Mary_e))'/<<e,t>,t> and '⟦[saw him]⟧ = (λ x_e: (λ y_e: Saw_<(e,e),t>(y_e, x_e)))'/<e,<e,t>> conflict (mode: Predicate Abstraction), ⟦[[saw him] mary2]⟧ = Type mismatch: '⟦[saw him]⟧ = (λ x_e: (λ y_e: Saw_<(e,e),t>(y_e, x_e)))'/<e,<e,t>>

In [14]:
(mary2 * (saw * him)).tree()

1 composition path:<br /><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{mary2}}]\!]^{}_{\langle{}\langle{}e,t\rangle{},t\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} f_{\langle{}e,t\rangle{}} \: . \: {f}_{\langle{}e,t\rangle{}}({Mary}_{e})$</div></div></td><td style="vertical-align:bottom;padding:10px">$\circ$</td><td style="vertical-align:bottom;padding:5px"><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{

This seems exactly right, with the abstraction over `x` corresponding to the intuition that `him` is free in this clause.

However, Jacobson points out the following problem with this account: what if the type of the name is not lifted?

In [15]:
mary1 * (saw * him)

CompositionResult(results=[⟦[[saw him] mary1]⟧ = (λ y_e: Saw_<(e,e),t>(y_e, Mary_e))], failures=[⟦[mary1 [saw him]]⟧ = Type mismatch: '⟦mary1⟧ = Mary_e'/e and '⟦[saw him]⟧ = (λ x_e: (λ y_e: Saw_<(e,e),t>(y_e, x_e)))'/<e,<e,t>> conflict (mode: Function Application), ⟦[mary1 [saw him]]⟧ = Type mismatch: '⟦mary1⟧ = Mary_e'/e and '⟦[saw him]⟧ = (λ x_e: (λ y_e: Saw_<(e,e),t>(y_e, x_e)))'/<e,<e,t>> conflict (mode: Predicate Modification), ⟦[mary1 [saw him]]⟧ = Type mismatch: '⟦mary1⟧ = Mary_e'/e and '⟦[saw him]⟧ = (λ x_e: (λ y_e: Saw_<(e,e),t>(y_e, x_e)))'/<e,<e,t>> conflict (mode: Predicate Abstraction), ⟦[[saw him] mary1]⟧ = Type mismatch: '⟦[saw him]⟧ = (λ x_e: (λ y_e: Saw_<(e,e),t>(y_e, x_e)))'/<e,<e,t>> and '⟦mary1⟧ = Mary_e'/e conflict (mode: Predicate Abstraction), ⟦[mary1 [saw him]]⟧ = Type mismatch: '(λ g_<I292,I291>: (λ f_<X,I292>: (λ x_X: g_<I292,I291>(f_<X,I292>(x_X)))))'/<<I292,I291>,<<X,I292>,<X,I291>>> and 'Mary_e'/e conflict (mode: Function argument combination (unification f

**Step 3**: rather than using a binary operation of function composition, use a unary type-shift that directly involves the geach combinator.

For a really general treatment of this we'd need a bunch more syntactic work (that Jacobson does), and a family of geach combinators.  I will just assume two geach combinators, and one lifting operation.

In [16]:
lift_combinator_t = %te L f_X : L g_<X,t> : g(f)
lift_combinator_t

(λ f_X: (λ g_<X,t>: g_<X,t>(f_X)))

In [17]:
lift_combinator_t(mary1.content).reduce()

(λ g_<e,t>: g_<e,t>(Mary_e))

In [18]:
g_e_combinator = geach.try_adjust_type(tp("<?,<<e,?>,?>>"))
g_e_combinator

(λ g_<?,Z>: (λ f_<e,?>: (λ x_e: g_<?,Z>(f_<e,?>(x_e)))))

In [19]:
g_et_combinator = geach.try_adjust_type(tp("<?,<<<e,t>,?>,?>>"))
g_et_combinator

(λ g_<?,Z>: (λ f_<<e,t>,?>: (λ x_<e,t>: g_<?,Z>(f_<<e,t>,?>(x_<e,t>)))))

In [20]:
def geach_shift(fun, f_type_left):
    combinator = geach_combinator(fun.type, types.FunType(f_type_left, fun.type.left))
    return combinator(fun).reduce_all()

In [21]:
geach_shift(f1, types.type_e)

NameError: global name 'geach_combinator' is not defined

In [22]:
system = lang.td_system.copy()
system.add_unary_rule(lift_combinator_t, "lift_t-shift")
system.add_unary_rule(g_e_combinator, "g_e-shift")
system.add_unary_rule(g_et_combinator, "g_et-shift")
lang.set_system(system)
system

Composition system: H&K simple (copy)

The brute-force way to trigger a unary type-shift is to multiply it by None (`x * None`).  The following examples use that idiom to show that, with the g-shift and type-lifted DP, there is no way to get the wrong result.

Though in principle the non-lifted subject can compose with the g-shifted verb, this is ruled out syntactically (not implemented here); abstracting function composition to the g-rule enables this.

In [23]:
(((mary2 * None) * ((saw * None) * him))).reduce_all().tree()

1 composition path:<br /><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{mary2}}]\!]^{}_{\langle{}\langle{}e,t\rangle{},t\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} f_{\langle{}e,t\rangle{}} \: . \: {f}_{\langle{}e,t\rangle{}}({Mary}_{e})$</div></div></td></tr></table></td><td style="border-left:1px solid #848482;vertical-align:center;padding:10px"><span style="color:blue"><b>[g_e-shift]</b></span></td></tr><tr style="border-style:solid;border-color:#848482;border-width:0px 1px 1px 1px"><td style="padding:5px" ali

To handle binding of pronouns, we add a new typeshift, the z-rule.  Again I will implement this using a combinator.

In [24]:
z_combinator = %te (λ f_<X,<e,Z>>: (λ g_<e,X>: (λ x_e: f(g(x))(x))))
z_combinator

(λ f_<X,<e,Z>>: (λ g_<e,X>: (λ x_e: f_<X,<e,Z>>(g_<e,X>(x_e))(x_e))))

In [25]:
z_combinator.try_adjust_type(tp("<<e,<e,t>>,?>"))

(λ f_<e,<e,t>>: (λ g_<e,e>: (λ x_e: f_<e,<e,t>>(g_<e,e>(x_e))(x_e))))

In [26]:
z_combinator(saw.content).reduce_all()

(λ g_<e,e>: (λ x_e: Saw_<(e,e),t>(x_e, g_<e,e>(x_e))))

In [27]:
system = lang.td_system.copy()
system.add_unary_rule(lift_combinator_t, "lift_t-shift")
system.add_typeshift(g_e_combinator, "g_e-shift")
system.add_typeshift(g_et_combinator, "g_et-shift")
system.add_typeshift(z_combinator, "z-shift")
system.typeshift=True
lang.set_system(system)
system


Composition system: H&K simple (copy)

In [28]:
r = mary2 * (saw * him)
r

Type-shifting: found 2 new items to try: [CompositionResult(results=[⟦[saw]⟧ = (λ f_<e,e>: (λ x_e: (λ y_e: Saw_<(e,e),t>(y_e, f_<e,e>(x_e))))), ⟦[saw]⟧ = (λ f_<<e,t>,e>: (λ x_<e,t>: (λ y_e: Saw_<(e,e),t>(y_e, f_<<e,t>,e>(x_<e,t>))))), ⟦[saw]⟧ = (λ g_<e,e>: (λ x_e: Saw_<(e,e),t>(x_e, g_<e,e>(x_e)))), ⟦saw⟧ = (λ x_e: (λ y_e: Saw_<(e,e),t>(y_e, x_e)))], failures=[]), CompositionResult(results=[⟦[him]⟧ = (λ f_<e,e>: (λ x_e: f_<e,e>(x_e))), ⟦[him]⟧ = (λ f_<<e,t>,e>: (λ x_<e,t>: f_<<e,t>,e>(x_<e,t>))), ⟦him⟧ = (λ x_e: x_e)], failures=[])]
Type-shifting: found 2 new items to try: [CompositionResult(results=[⟦[mary2]⟧ = (λ f_<e,<e,t>>: (λ x_e: f_<e,<e,t>>(x_e)(Mary_e))), ⟦[mary2]⟧ = (λ f_<<e,t>,<e,t>>: (λ x_<e,t>: f_<<e,t>,<e,t>>(x_<e,t>)(Mary_e))), ⟦mary2⟧ = (λ f_<e,t>: f_<e,t>(Mary_e))], failures=[]), CompositionResult(results=[⟦[[[saw] him]]⟧ = (λ f_<e,e>: (λ x_e: (λ y_e: Saw_<(e,e),t>(y_e, f_<e,e>(x_e))))), ⟦[[[saw] him]]⟧ = (λ f_<<e,t>,e>: (λ x_<e,t>: (λ y_e: Saw_<(e,e),t>(y_e, f_<<e,t>,e

CompositionResult(results=[⟦[[mary2] [[saw] him]]⟧ = (λ x_e: Saw_<(e,e),t>(Mary_e, x_e)), ⟦[mary2 [[saw] him]]⟧ = Saw_<(e,e),t>(Mary_e, Mary_e)], failures=[⟦[[mary2] [[[saw] him]]]⟧ = Type mismatch: '⟦[mary2]⟧ = (λ f_<e,<e,t>>: (λ x_e: f_<e,<e,t>>(x_e)(Mary_e)))'/<<e,<e,t>>,<e,t>> and '⟦[[[saw] him]]⟧ = (λ f_<e,e>: (λ x_e: (λ y_e: Saw_<(e,e),t>(y_e, f_<e,e>(x_e)))))'/<<e,e>,<e,<e,t>>> conflict (mode: Function Application), ⟦[[[[saw] him]] [mary2]]⟧ = Type mismatch: '⟦[[[saw] him]]⟧ = (λ f_<e,e>: (λ x_e: (λ y_e: Saw_<(e,e),t>(y_e, f_<e,e>(x_e)))))'/<<e,e>,<e,<e,t>>> and '⟦[mary2]⟧ = (λ f_<e,<e,t>>: (λ x_e: f_<e,<e,t>>(x_e)(Mary_e)))'/<<e,<e,t>>,<e,t>> conflict (mode: Function Application), ⟦[[mary2] [[[saw] him]]]⟧ = Type mismatch: '⟦[mary2]⟧ = (λ f_<e,<e,t>>: (λ x_e: f_<e,<e,t>>(x_e)(Mary_e)))'/<<e,<e,t>>,<e,t>> and '⟦[[[saw] him]]⟧ = (λ f_<e,e>: (λ x_e: (λ y_e: Saw_<(e,e),t>(y_e, f_<e,e>(x_e)))))'/<<e,e>,<e,<e,t>>> conflict (mode: Predicate Modification), ⟦[[mary2] [[[saw] him]]]⟧ = T

Note that `r[1]` is effectively a reflexive reading, which is a condition B violation for the pronoun *him*.  That is, this reading is generated by having the subject bind the direct object (via the z-rule).  Nothing in the system at the moment rules this out, and so these typeshifts alone will overgenerate reflexive readings.

`r[0]` is of course the reading we hoped to generate, where `him` is free.

In [29]:
r[0].tree()

In [30]:
r[1].tree()

In [31]:
%%lamb
||every|| = L f_<e,t> : L g_<e,t> : Forall x_e : f(x) >> g(x)
||man|| = L x_e : Man(x)

INFO (meta): Coerced guessed type t for 'Man_t' into <e,t>, to match argument 'x_e'


$[\![\mathbf{\text{every}}]\!]^{}_{\langle{}\langle{}e,t\rangle{},\langle{}\langle{}e,t\rangle{},t\rangle{}\rangle{}} \:=\: \lambda{} f_{\langle{}e,t\rangle{}} \: . \: \lambda{} g_{\langle{}e,t\rangle{}} \: . \: \forall{} x_{e} \: . \: ({f}_{\langle{}e,t\rangle{}}({x}_{e}) \rightarrow{} {g}_{\langle{}e,t\rangle{}}({x}_{e}))$<br />
$[\![\mathbf{\text{man}}]\!]^{}_{\langle{}e,t\rangle{}} \:=\: \lambda{} x_{e} \: . \: {Man}({x}_{e})$

In [32]:
dp = every * man
dp

CompositionResult(results=[⟦[every man]⟧ = (λ g_<e,t>: (Forall x_e: (Man_<e,t>(x_e) >> g_<e,t>(x_e))))], failures=[⟦[man every]⟧ = Type mismatch: '⟦man⟧ = (λ x_e: Man_<e,t>(x_e))'/<e,t> and '⟦every⟧ = (λ f_<e,t>: (λ g_<e,t>: (Forall x_e: (f_<e,t>(x_e) >> g_<e,t>(x_e)))))'/<<e,t>,<<e,t>,t>> conflict (mode: Function Application), ⟦[every man]⟧ = Type mismatch: '⟦every⟧ = (λ f_<e,t>: (λ g_<e,t>: (Forall x_e: (f_<e,t>(x_e) >> g_<e,t>(x_e)))))'/<<e,t>,<<e,t>,t>> and '⟦man⟧ = (λ x_e: Man_<e,t>(x_e))'/<e,t> conflict (mode: Predicate Modification), ⟦[every man]⟧ = Type mismatch: '⟦every⟧ = (λ f_<e,t>: (λ g_<e,t>: (Forall x_e: (f_<e,t>(x_e) >> g_<e,t>(x_e)))))'/<<e,t>,<<e,t>,t>> and '⟦man⟧ = (λ x_e: Man_<e,t>(x_e))'/<e,t> conflict (mode: Predicate Abstraction), ⟦[man every]⟧ = Type mismatch: '⟦man⟧ = (λ x_e: Man_<e,t>(x_e))'/<e,t> and '⟦every⟧ = (λ f_<e,t>: (λ g_<e,t>: (Forall x_e: (f_<e,t>(x_e) >> g_<e,t>(x_e)))))'/<<e,t>,<<e,t>,t>> conflict (mode: Predicate Abstraction)])

In [33]:
(dp * (saw * him))

Type-shifting: found 2 new items to try: [CompositionResult(results=[⟦[saw]⟧ = (λ f_<e,e>: (λ x_e: (λ y_e: Saw_<(e,e),t>(y_e, f_<e,e>(x_e))))), ⟦[saw]⟧ = (λ f_<<e,t>,e>: (λ x_<e,t>: (λ y_e: Saw_<(e,e),t>(y_e, f_<<e,t>,e>(x_<e,t>))))), ⟦[saw]⟧ = (λ g_<e,e>: (λ x_e: Saw_<(e,e),t>(x_e, g_<e,e>(x_e)))), ⟦saw⟧ = (λ x_e: (λ y_e: Saw_<(e,e),t>(y_e, x_e)))], failures=[]), CompositionResult(results=[⟦[him]⟧ = (λ f_<e,e>: (λ x_e: f_<e,e>(x_e))), ⟦[him]⟧ = (λ f_<<e,t>,e>: (λ x_<e,t>: f_<<e,t>,e>(x_<e,t>))), ⟦him⟧ = (λ x_e: x_e)], failures=[])]
Type-shifting: found 2 new items to try: [CompositionResult(results=[⟦[[every man]]⟧ = (λ f_<e,<e,t>>: (λ x_e: (Forall x1_e: (Man_<e,t>(x1_e) >> f_<e,<e,t>>(x_e)(x1_e))))), ⟦[[every man]]⟧ = (λ f_<<e,t>,<e,t>>: (λ x_<e,t>: (Forall x1_e: (Man_<e,t>(x1_e) >> f_<<e,t>,<e,t>>(x_<e,t>)(x1_e))))), ⟦[every man]⟧ = (λ g_<e,t>: (Forall x_e: (Man_<e,t>(x_e) >> g_<e,t>(x_e))))], failures=[]), CompositionResult(results=[⟦[[[saw] him]]⟧ = (λ f_<e,e>: (λ x_e: (λ y_e: Saw

CompositionResult(results=[⟦[[[every man]] [[saw] him]]⟧ = (λ x_e: (Forall x1_e: (Man_<e,t>(x1_e) >> Saw_<(e,e),t>(x1_e, x_e)))), ⟦[[every man] [[saw] him]]⟧ = (Forall x_e: (Man_<e,t>(x_e) >> Saw_<(e,e),t>(x_e, x_e)))], failures=[⟦[[[every man]] [[[saw] him]]]⟧ = Type mismatch: '⟦[[every man]]⟧ = (λ f_<e,<e,t>>: (λ x_e: (Forall x1_e: (Man_<e,t>(x1_e) >> f_<e,<e,t>>(x_e)(x1_e)))))'/<<e,<e,t>>,<e,t>> and '⟦[[[saw] him]]⟧ = (λ f_<e,e>: (λ x_e: (λ y_e: Saw_<(e,e),t>(y_e, f_<e,e>(x_e)))))'/<<e,e>,<e,<e,t>>> conflict (mode: Function Application), ⟦[[[[saw] him]] [[every man]]]⟧ = Type mismatch: '⟦[[[saw] him]]⟧ = (λ f_<e,e>: (λ x_e: (λ y_e: Saw_<(e,e),t>(y_e, f_<e,e>(x_e)))))'/<<e,e>,<e,<e,t>>> and '⟦[[every man]]⟧ = (λ f_<e,<e,t>>: (λ x_e: (Forall x1_e: (Man_<e,t>(x1_e) >> f_<e,<e,t>>(x_e)(x1_e)))))'/<<e,<e,t>>,<e,t>> conflict (mode: Function Application), ⟦[[[every man]] [[[saw] him]]]⟧ = Type mismatch: '⟦[[every man]]⟧ = (λ f_<e,<e,t>>: (λ x_e: (Forall x1_e: (Man_<e,t>(x1_e) >> f_<e,<e,t>

Once again, we overgenerate a condition B violation reading.  However, this process generates exactly the right readings for bound pronouns not in the local domain of the binder, as Jacobson shows:

In [34]:
%%lamb
||poss|| = L f_<e,t> : Iota x_e : f(x) # just definite article
||mother|| = L x_e : L y_e : Mother(y,x)

INFO (meta): Coerced guessed type t for 'Mother_t' into <(e,e),t>, to match argument '(y_e, x_e)'


$[\![\mathbf{\text{poss}}]\!]^{}_{\langle{}\langle{}e,t\rangle{},e\rangle{}} \:=\: \lambda{} f_{\langle{}e,t\rangle{}} \: . \: \iota{} x_{e} \: . \: {f}_{\langle{}e,t\rangle{}}({x}_{e})$<br />
$[\![\mathbf{\text{mother}}]\!]^{}_{\langle{}e,\langle{}e,t\rangle{}\rangle{}} \:=\: \lambda{} x_{e} \: . \: \lambda{} y_{e} \: . \: {Mother}({y}_{e}, {x}_{e})$

In [44]:
%prun -s cumulative result = (dp * (saw * (poss * (mother * him))))
result

Type-shifting: found 2 new items to try: [CompositionResult(results=[⟦[mother]⟧ = (λ f_<e,e>: (λ x_e: (λ y_e: Mother_<(e,e),t>(y_e, f_<e,e>(x_e))))), ⟦[mother]⟧ = (λ f_<<e,t>,e>: (λ x_<e,t>: (λ y_e: Mother_<(e,e),t>(y_e, f_<<e,t>,e>(x_<e,t>))))), ⟦[mother]⟧ = (λ g_<e,e>: (λ x_e: Mother_<(e,e),t>(x_e, g_<e,e>(x_e)))), ⟦mother⟧ = (λ x_e: (λ y_e: Mother_<(e,e),t>(y_e, x_e)))], failures=[]), CompositionResult(results=[⟦[him]⟧ = (λ f_<e,e>: (λ x_e: f_<e,e>(x_e))), ⟦[him]⟧ = (λ f_<<e,t>,e>: (λ x_<e,t>: f_<<e,t>,e>(x_<e,t>))), ⟦him⟧ = (λ x_e: x_e)], failures=[])]
Type-shifting: found 2 new items to try: [CompositionResult(results=[⟦[poss]⟧ = (λ f_<e,<e,t>>: (λ x_e: (ι x1_e: f_<e,<e,t>>(x_e)(x1_e)))), ⟦[poss]⟧ = (λ f_<<e,t>,<e,t>>: (λ x_<e,t>: (ι x1_e: f_<<e,t>,<e,t>>(x_<e,t>)(x1_e)))), ⟦poss⟧ = (λ f_<e,t>: (ι x_e: f_<e,t>(x_e)))], failures=[]), CompositionResult(results=[⟦[[[mother] him]]⟧ = (λ f_<e,e>: (λ x_e: (λ y_e: Mother_<(e,e),t>(y_e, f_<e,e>(x_e))))), ⟦[[[mother] him]]⟧ = (λ f_<<e,t>,e

CompositionResult(results=[⟦[[[every man]] [[saw] [[poss] [[mother] him]]]]⟧ = (λ x_e: (Forall x1_e: (Man_<e,t>(x1_e) >> Saw_<(e,e),t>(x1_e, (ι x2_e: Mother_<(e,e),t>(x2_e, x_e)))))), ⟦[[every man] [[saw] [[poss] [[mother] him]]]]⟧ = (Forall x_e: (Man_<e,t>(x_e) >> Saw_<(e,e),t>(x_e, (ι x1_e: Mother_<(e,e),t>(x1_e, x_e))))), ⟦[[[every man]] [[[[[[mother] him]] poss]] saw]]⟧ = (λ x_e: (Forall x1_e: (Man_<e,t>(x1_e) >> Mother_<(e,e),t>(x1_e, (ι x2_e: Saw_<(e,e),t>(x2_e, x_e)))))), ⟦[[every man] [[[[[[mother] him]] poss]] saw]]⟧ = (Forall x_e: (Man_<e,t>(x_e) >> Mother_<(e,e),t>(x_e, (ι x1_e: Saw_<(e,e),t>(x1_e, x_e))))), ⟦[[every man] [saw [poss [[mother] him]]]]⟧ = (Forall x_e: (Man_<e,t>(x_e) >> Saw_<(e,e),t>(x_e, (ι x1_e: Mother_<(e,e),t>(x1_e, x1_e)))))], failures=[⟦[[[every man]] [[[saw] [[poss] [[mother] him]]]]]⟧ = Type mismatch: '⟦[[every man]]⟧ = (λ f_<e,<e,t>>: (λ x_e: (Forall x1_e: (Man_<e,t>(x1_e) >> f_<e,<e,t>>(x_e)(x1_e)))))'/<<e,<e,t>>,<e,t>> and '⟦[[[saw] [[poss] [[mother

In [43]:
%prun -s cumtime (mother * him)

Type-shifting: found 2 new items to try: [CompositionResult(results=[⟦[mother]⟧ = (λ f_<e,e>: (λ x_e: (λ y_e: Mother_<(e,e),t>(y_e, f_<e,e>(x_e))))), ⟦[mother]⟧ = (λ f_<<e,t>,e>: (λ x_<e,t>: (λ y_e: Mother_<(e,e),t>(y_e, f_<<e,t>,e>(x_<e,t>))))), ⟦[mother]⟧ = (λ g_<e,e>: (λ x_e: Mother_<(e,e),t>(x_e, g_<e,e>(x_e)))), ⟦mother⟧ = (λ x_e: (λ y_e: Mother_<(e,e),t>(y_e, x_e)))], failures=[]), CompositionResult(results=[⟦[him]⟧ = (λ f_<e,e>: (λ x_e: f_<e,e>(x_e))), ⟦[him]⟧ = (λ f_<<e,t>,e>: (λ x_<e,t>: f_<<e,t>,e>(x_<e,t>))), ⟦him⟧ = (λ x_e: x_e)], failures=[])]
 

This account generates 3 readings, 2 of them redundant (but generated via different sequences of typeshifts).  (*note, numbers may not be stable across ILNB version changes*) 

 * `result[1]` and `result[3]` are the (same) bound variable reading that is exactly what we are looking for.
 * `result[0]` and `result[2]` are the (same) free reading that we also want.
 * `result[4]` is an overgenerated, and pathological, reflexive reading where `x1` is `x1`'s own mother.  (*TODO: treatment of possessive is slightly different than Jacobson's...*)

In [37]:
result[1].tree()

In [38]:
system

Composition system: H&K simple (copy)

In [39]:
system.rules[5].operation

<function lamb.lang.unary_factory.<locals>.op_fun>

In [40]:
g_e_combinator._type_adjust_cache

{<<<e,t>,t>,<<e,<e,t>>,<e,t>>>: (λ g_<<e,t>,t>: (λ f_<e,<e,t>>: (λ x_e: g_<<e,t>,t>(f_<e,<e,t>>(x_e))))),
 <<e,<e,t>>,<<e,e>,<e,<e,t>>>>: (λ g_<e,<e,t>>: (λ f_<e,e>: (λ x_e: g_<e,<e,t>>(f_<e,e>(x_e))))),
 <<<e,t>,<e,t>>,<<e,<e,t>>,<e,<e,t>>>>: (λ g_<<e,t>,<e,t>>: (λ f_<e,<e,t>>: (λ x_e: g_<<e,t>,<e,t>>(f_<e,<e,t>>(x_e))))),
 <<e,e>,<<e,e>,<e,e>>>: (λ g_<e,e>: (λ f_<e,e>: (λ x_e: g_<e,e>(f_<e,e>(x_e))))),
 <<<e,t>,e>,<<e,<e,t>>,<e,e>>>: (λ g_<<e,t>,e>: (λ f_<e,<e,t>>: (λ x_e: g_<<e,t>,e>(f_<e,<e,t>>(x_e)))))}

In [41]:
%prun geach(poss.content)

In [49]:
%time lamb.combinators.lift(man.content, types.type_t)

CPU times: user 2.39 ms, sys: 32 µs, total: 2.43 ms
Wall time: 2.42 ms


(λ g_<<e,t>,t>: g_<<e,t>,t>((λ x_e: Man_<e,t>(x_e))))

In [55]:
%time lamb.combinators.lift_combinator_t(man.content).reduce_all()

CPU times: user 7.06 ms, sys: 300 µs, total: 7.36 ms
Wall time: 7.1 ms


(λ g_<<e,t>,t>: g_<<e,t>,t>((λ x_e: Man_<e,t>(x_e))))

In [59]:
%time combinators.build_geach_combinator(tp("<e,t>"), tp("<e,e>"))(man.content).reduce_all()

CPU times: user 4.41 ms, sys: 135 µs, total: 4.54 ms
Wall time: 4.48 ms


(λ f_<e,e>: (λ x_e: Man_<e,t>(f_<e,e>(x_e))))

In [62]:
%time geach(man.content) #.reduce_all()

CPU times: user 11 ms, sys: 370 µs, total: 11.4 ms
Wall time: 11.1 ms


((λ g_<e,t>: (λ f_<X,e>: (λ x_X: g_<e,t>(f_<X,e>(x_X))))))((λ x_e: Man_<e,t>(x_e)))